##### Copyright 2021 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Copyright 2021 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/hub/tutorials/wav2vec2_saved_model_finetuning"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/hub/tutorials/wav2vec2_saved_model_finetuning.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/hub/tutorials/wav2vec2_saved_model_finetuning.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/hub/tutorials/wav2vec2_saved_model_finetuning.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
  <td>     <a href="https://tfhub.dev/vasudevgupta7/wav2vec2/1"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">Ver modelos de TF Hub</a>
</td>
</table>

# Ajuste de Wav2Vec2 con un cabezal de modelado de lenguaje

En este cuaderno, cargaremos el modelo wav2vec2 preentrenado desde [TFHub](https://tfhub.dev). Para ajustarlo en el [conjunto de datos LibriSpeech](https://huggingface.co/datasets/librispeech_asr), agregaremos el cabezal de modelado de lenguaje (LM) en la parte superior de nuestro modelo preentrenado. La tarea subyacente es generar un modelo para el **reconocimiento de voz automático**, es decir, a partir de ingresar algo mediante la voz, el modelo debería poder transcribirlo a texto.

## Configuración

Antes de ejecutar este cuaderno, asegúrese de estar en tiempo de ejecución de GPU (`Runtime` &gt; `Change runtime type` &gt; `GPU`). La siguiente celda instalará el paquete [`gsoc-wav2vec2`](https://github.com/vasudevgupta7/gsoc-wav2vec2) y sus dependencias.

In [ ]:
!pip3 install -q git+https://github.com/vasudevgupta7/gsoc-wav2vec2@main
!sudo apt-get install -y libsndfile1-dev
!pip3 install -q SoundFile

## Preparar el modelo con `TFHub`

Comenzaremos por importar algunas bibliotecas/módulos.

In [ ]:
import os

import tensorflow as tf
import tensorflow_hub as hub
from wav2vec2 import Wav2Vec2Config

config = Wav2Vec2Config()

print("TF version:", tf.__version__)

Primero, descargaremos nuestro modelo de TFHub y envolveremos la signatura de nuestro modelo con [`hub.KerasLayer`](https://www.tensorflow.org/hub/api_docs/python/hub/KerasLayer) para poder usar este modelo como cualquier otra capa de Keras. Por suerte, `hub.KerasLayer` puede hacer ambas cosas en 1 sola línea.

**Nota:** Al cargar el modelo con `hub.KerasLayer`, el modelo se vuelve un poco opaco pero a veces necesitamos controles más precisos sobre el modelo, luego podemos cargar el modelo con `tf.keras.models.load_model(...)`.

In [ ]:
pretrained_layer = hub.KerasLayer("https://tfhub.dev/vasudevgupta7/wav2vec2/1", trainable=True)

Puede consultar este [script](https://github.com/vasudevgupta7/gsoc-wav2vec2/blob/main/src/export2hub.py) en caso de que le interese el script de exportación del modelo. El objeto `pretrained_layer` es la versión inmovilizada de [`Wav2Vec2Model`](https://github.com/vasudevgupta7/gsoc-wav2vec2/blob/main/src/wav2vec2/modeling.py). Estos pesos preentrenados se convirtieron a partir de [pesos preentrenados en](https://huggingface.co/facebook/wav2vec2-base) HuggingFace PyTorch con [este script](https://github.com/vasudevgupta7/gsoc-wav2vec2/blob/main/src/convert_torch_to_tf.py).

Originalmente, wav2vec2 se preentrenó con un enfoque de modelado de lenguaje enmascarado con el objetivo de identificar la verdadera representación del habla latente cuantificada durante un paso de tiempo enmascarado. Puede leer más sobre el objetivo del entrenamiento en el artículo [wav2vec 2.0: Un marco para el aprendizaje autosupervisado de representaciones del habla](https://arxiv.org/abs/2006.11477).

Ahora definiremos algunas constantes e hiperparámetros que serán útiles en las siguientes celdas. `AUDIO_MAXLEN` se establece intencionalmente en `246000` ya que la signatura del modelo solo acepta una longitud de secuencia estática de `246000`.

In [ ]:
AUDIO_MAXLEN = 246000
LABEL_MAXLEN = 256
BATCH_SIZE = 2

En la siguiente celda, envolveremos `pretrained_layer` y una capa densa (cabezal del modelado de lenguaje) con la [API funcional de Keras](https://www.tensorflow.org/guide/keras/functional).

In [ ]:
inputs = tf.keras.Input(shape=(AUDIO_MAXLEN,))
hidden_states = pretrained_layer(inputs)
outputs = tf.keras.layers.Dense(config.vocab_size)(hidden_states)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

La capa densa (definida anteriormente) tiene una dimensión de salida de `vocab_size`, ya que queremos predecir las probabilidades de cada token en el vocabulario en cada paso de tiempo.

## Configurar el estado de entrenamiento

En TensorFlow, los pesos del modelo se crean solo cuando se llama `model.call` o `model.build` por primera vez, por lo que la siguiente celda creará los pesos del modelo por nosotros. Además, ejecutaremos `model.summary()` para verificar la cantidad total de parámetros entrenables.

In [ ]:
model(tf.random.uniform(shape=(BATCH_SIZE, AUDIO_MAXLEN)))
model.summary()

Ahora, necesitamos definir `loss_fn` y el optimizador para poder entrenar el modelo. La siguiente celda hará eso por nosotros. Usaremos el optimizador `Adam` por simplicidad. `CTCLoss` es un tipo de pérdida común que se usa para tareas (como `ASR`) donde las subpartes de entrada no se pueden alinear fácilmente con las subpartes de salida. Puede leer más sobre la pérdida de CTC en esta increíble [publicación de blog](https://distill.pub/2017/ctc/).

`CTCLoss` (del paquete [`gsoc-wav2vec2`](https://github.com/vasudevgupta7/gsoc-wav2vec2)) acepta 3 argumentos: `config`, `model_input_shape` y `division_factor`. Si `division_factor=1`, entonces la pérdida simplemente se sumará, así que pase `division_factor` en consecuencia para obtener la media sobre el lote.

In [ ]:
from wav2vec2 import CTCLoss

LEARNING_RATE = 5e-5

loss_fn = CTCLoss(config, (BATCH_SIZE, AUDIO_MAXLEN), division_factor=BATCH_SIZE)
optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)

## Cargar y preprocesar los datos

Ahora descargaremos el conjunto de datos LibriSpeech del [sitio web oficial](http://www.openslr.org/12) y lo configuraremos.

In [ ]:
!wget https://www.openslr.org/resources/12/dev-clean.tar.gz -P ./data/train/
!tar -xf ./data/train/dev-clean.tar.gz -C ./data/train/

**Nota:** Estamos usando la configuración `dev-clean` ya que este cuaderno es solo para fines de demostración, por lo que necesitamos una pequeña cantidad de datos. Los datos completos del entrenamiento se pueden descargar fácilmente desde [el sitio web de LibriSpeech](http://www.openslr.org/12).

In [ ]:
ls ./data/train/

Nuestro conjunto de datos se encuentra en el directorio LibriSpeech. Exploremos estos archivos.

In [ ]:
data_dir = "./data/train/LibriSpeech/dev-clean/2428/83705/"
all_files = os.listdir(data_dir)

flac_files = [f for f in all_files if f.endswith(".flac")]
txt_files = [f for f in all_files if f.endswith(".txt")]

print("Transcription files:", txt_files, "\nSound files:", flac_files)

Muy bien, entonces cada subdirectorio tiene muchos archivos `.flac` y un archivo `.txt`. El archivo `.txt` contiene transcripciones de texto para todas las muestras de voz (es decir, archivos `.flac`) presentes en ese subdirectorio.

Podemos cargar estos datos de texto de la siguiente manera:

In [ ]:
def read_txt_file(f):
  with open(f, "r") as f:
    samples = f.read().split("\n")
    samples = {s.split()[0]: " ".join(s.split()[1:]) for s in samples if len(s.split()) > 2}
  return samples

De manera similar, definiremos una función para cargar una muestra de voz desde un archivo `.flac`.

`REQUIRED_SAMPLE_RATE` está configurado en `16000` ya que wav2vec2 fue preentrenado con una frecuencia `16K` y se recomienda ajustarlo sin ningún cambio importante en la distribución de datos debido a la frecuencia.

In [ ]:
import soundfile as sf

REQUIRED_SAMPLE_RATE = 16000

def read_flac_file(file_path):
  with open(file_path, "rb") as f:
      audio, sample_rate = sf.read(f)
  if sample_rate != REQUIRED_SAMPLE_RATE:
      raise ValueError(
          f"sample rate (={sample_rate}) of your files must be {REQUIRED_SAMPLE_RATE}"
      )
  file_id = os.path.split(file_path)[-1][:-len(".flac")]
  return {file_id: audio}

Ahora, elegiremos algunas muestras aleatorias e intentaremos visualizarlas.

In [ ]:
from IPython.display import Audio
import random

file_id = random.choice([f[:-len(".flac")] for f in flac_files])
flac_file_path, txt_file_path = os.path.join(data_dir, f"{file_id}.flac"), os.path.join(data_dir, "2428-83705.trans.txt")

print("Text Transcription:", read_txt_file(txt_file_path)[file_id], "\nAudio:")
Audio(filename=flac_file_path)

Ahora, combinaremos todas las muestras de voz y texto y definiremos la función (en la siguiente celda) para ese propósito.

In [ ]:
def fetch_sound_text_mapping(data_dir):
  all_files = os.listdir(data_dir)

  flac_files = [os.path.join(data_dir, f) for f in all_files if f.endswith(".flac")]
  txt_files = [os.path.join(data_dir, f) for f in all_files if f.endswith(".txt")]

  txt_samples = {}
  for f in txt_files:
    txt_samples.update(read_txt_file(f))

  speech_samples = {}
  for f in flac_files:
    speech_samples.update(read_flac_file(f))

  assert len(txt_samples) == len(speech_samples)

  samples = [(speech_samples[file_id], txt_samples[file_id]) for file_id in speech_samples.keys() if len(speech_samples[file_id]) < AUDIO_MAXLEN]
  return samples

Llegó la hora de ver algunas muestras...

In [ ]:
samples = fetch_sound_text_mapping(data_dir)
samples[:5]

Nota: Estamos cargando estos datos en la memoria mientras trabajamos con una pequeña cantidad de conjunto de datos en este cuaderno. Pero para entrenar con el conjunto de datos completo (300 GB apróx.), tendrá que cargar los datos de forma diferida. Puede consultar [este script](https://github.com/vasudevgupta7/gsoc-wav2vec2/blob/main/src/data_utils.py) para obtener más información al respecto.

¡Preprocesemos los datos ya!

Primero definiremos el tokenizador y el procesador con el paquete `gsoc-wav2vec2`. Luego, haremos un preprocesamiento muy simple. `processor` normalizará el eje de fotogramas en relación con la voz sin procesar y `tokenizer` convertirá las salidas de nuestro modelo en la cadena de texto (usando el vocabulario definido) y se encargará de eliminar los tokens especiales (según la configuración de su tokenizador).

In [ ]:
from wav2vec2 import Wav2Vec2Processor
tokenizer = Wav2Vec2Processor(is_tokenizer=True)
processor = Wav2Vec2Processor(is_tokenizer=False)

def preprocess_text(text):
  label = tokenizer(text)
  return tf.constant(label, dtype=tf.int32)

def preprocess_speech(audio):
  audio = tf.constant(audio, dtype=tf.float32)
  return processor(tf.transpose(audio))

Ahora, definiremos el generador de Python para llamar a las funciones de preprocesamiento que definimos en las celdas anteriores.

In [ ]:
def inputs_generator():
  for speech, text in samples:
    yield preprocess_speech(speech), preprocess_text(text)

## Configurar `tf.data.Dataset`

En la siguiente celda configuraremos el objeto `tf.data.Dataset` con su método `.from_generator(...)`. Usaremos el objeto `generator` que definimos en la celda anterior.

**Nota:** Para el entrenamiento distribuido (especialmente en las unidades de procesamiento de tensores [TPU por sus siglas en inglés]), `.from_generator(...)` no funciona actualmente y se recomienda entrenar con datos almacenados en formato `.tfrecord`. (Nota: Lo ideal es que los archivos  TFRecords se almacenen dentro de un depósito de Google Cloud Storage para que las TPU funcionen al máximo).

Puede consultar [este script](https://github.com/vasudevgupta7/gsoc-wav2vec2/blob/main/src/make_tfrecords.py) para obtener más detalles sobre cómo convertir datos de LibriSpeech en tfrecords.

In [ ]:
output_signature = (
    tf.TensorSpec(shape=(None),  dtype=tf.float32),
    tf.TensorSpec(shape=(None), dtype=tf.int32),
)

dataset = tf.data.Dataset.from_generator(inputs_generator, output_signature=output_signature)

In [ ]:
BUFFER_SIZE = len(flac_files)
SEED = 42

dataset = dataset.shuffle(BUFFER_SIZE, seed=SEED)

Pasaremos el conjunto de datos a varios lotes, así que preparemos los lotes en la siguiente celda. Ahora, todas las secuencias de un lote deben rellenarse hasta una longitud constante. Usaremos el método `.padded_batch(...)` para ese propósito.

In [ ]:
dataset = dataset.padded_batch(BATCH_SIZE, padded_shapes=(AUDIO_MAXLEN, LABEL_MAXLEN), padding_values=(0.0, 0))

Los aceleradores (como GPU/TPU) son muy rápidos y, a menudo, la carga de datos (y el preprocesamiento) se convierte en un cuello de botella durante el entrenamiento, ya que la parte de carga de datos ocurre en las CPU. Esto puede aumentar significativamente el tiempo de entrenamiento, especialmente cuando se necesita mucho preprocesamiento en línea o los datos se transmiten en línea desde depósitos de GCS. Para tratar estos problemas, `tf.data.Dataset` ofrece el método `.prefetch(...)`. Este método ayuda a preparar los siguientes lotes en paralelo (en CPU) mientras el modelo realiza predicciones (en GPU/TPU) en el lote actual.

In [ ]:
dataset = dataset.prefetch(tf.data.AUTOTUNE)

Dado que este cuaderno está creado con fines de demostración, tomaremos primero `num_train_batches` y solo lo entrenaremos para eso. Sin embargo, le recomendamos entrenar  todo el conjunto de datos. De manera similar, evaluaremos solo `num_val_batches`.

In [ ]:
num_train_batches = 10
num_val_batches = 4

train_dataset = dataset.take(num_train_batches)
val_dataset = dataset.skip(num_train_batches).take(num_val_batches)

## Entrenamiento del modelo

Para entrenar nuestro modelo, llamaremos directamente al método `.fit(...)` después de compilar nuestro modelo con `.compile(...)`.

In [ ]:
model.compile(optimizer, loss=loss_fn)

La celda anterior configurará nuestro estado de entrenamiento. Ahora podemos iniciar el entrenamiento con el método `.fit(...)`.

In [ ]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs=3)
history.history

Guardemos nuestro modelo con el método `.save(...)` para poder realizar la inferencia más adelante. También puede exportar este SavedModel a TFHub siguiendo los pasos de [la documentación de TFHub](https://www.tensorflow.org/hub/publish).

In [ ]:
save_dir = "finetuned-wav2vec2"
model.save(save_dir, include_optimizer=False)

Nota: Estamos configurando `include_optimizer=False` porque queremos usar este modelo solo para inferencias.

## Evaluación

Ahora calcularemos la tasa de error de palabras sobre el conjunto de datos de validación.

**La tasa de error de palabras** (WER por sus siglas en inglés) es una métrica común para medir el rendimiento de un sistema de reconocimiento de voz automático. El WER se deriva de la distancia de Levenshtein, trabajando a nivel de palabra. La tasa de error de palabra se puede calcular como: WER = (S + D + I) / N = (S + D + I) / (S + D + C) donde S es el número de sustituciones, D es el número de eliminaciones, I es el número de inserciones, C es el número de palabras correctas, N es el número de palabras en la referencia (N=S+D+C). Este valor indica el porcentaje de palabras que se predijeron incorrectamente.

Puede consultar [este documento](https://www.isca-speech.org/archive_v0/interspeech_2004/i04_2765.html) para obtener más información sobre WER.

Usaremos la función `load_metric(...)` de la biblioteca de [conjuntos de datos HuggingFace](https://huggingface.co/docs/datasets/). Primero instalemos la biblioteca de `datasets` con `pip` y luego definamos el objeto `metric`.

In [ ]:
!pip3 install -q datasets

from datasets import load_metric
metric = load_metric("wer")

In [ ]:
@tf.function(jit_compile=True)
def eval_fwd(batch):
  logits = model(batch, training=False)
  return tf.argmax(logits, axis=-1)

Ya podemos ejecutar la evaluación de los datos de validación.

In [ ]:
from tqdm.auto import tqdm

for speech, labels in tqdm(val_dataset, total=num_val_batches):
    predictions  = eval_fwd(speech)
    predictions = [tokenizer.decode(pred) for pred in predictions.numpy().tolist()]
    references = [tokenizer.decode(label, group_tokens=False) for label in labels.numpy().tolist()]
    metric.add_batch(references=references, predictions=predictions)

Estamos usando el método `tokenizer.decode(...)` para volver a decodificar nuestras predicciones y etiquetas en el texto y las agregaremos a la métrica para el cálculo `WER` más adelante.

Ahora, calculemos el valor de la métrica en la siguiente celda:

In [ ]:
metric.compute()

**Nota:** Aquí el valor de la métrica no tiene ningún sentido ya que el modelo se entrena con datos muy pequeños y las tareas como las de reconocimiento de voz automático (ASR por sus siglas en inglés) a menudo requieren una gran cantidad de datos para aprender una asignación de voz a texto. Probablemente debería entrenar con una gran cantidad de datos para obtener buenos resultados. Este cuaderno le proporciona una plantilla para ajustar un modelo de voz preentrenado.

## Inferencia

Ahora que estamos satisfechos con el proceso de entrenamiento y hemos guardado el modelo en `save_dir`, veremos cómo se puede usar este modelo para la inferencia.

Primero, cargaremos nuestro modelo con `tf.keras.models.load_model(...)`.

In [ ]:
finetuned_model = tf.keras.models.load_model(save_dir)

Descarguemos algunas muestras de voz para realizar inferencias. También puede reemplazar la siguiente muestra con su muestra de voz.

In [ ]:
!wget https://github.com/vasudevgupta7/gsoc-wav2vec2/raw/main/data/SA2.wav

Ahora, leeremos la muestra de voz con `soundfile.read(...)` y la rellenaremos con `AUDIO_MAXLEN` para satisfacer la signatura del modelo. Luego normalizaremos esa muestra de voz con la instancia `Wav2Vec2Processor` y la ingresaremos en el modelo.

In [ ]:
import numpy as np

speech, _ = sf.read("SA2.wav")
speech = np.pad(speech, (0, AUDIO_MAXLEN - len(speech)))
speech = tf.expand_dims(processor(tf.constant(speech)), 0)

outputs = finetuned_model(speech)
outputs

Decodifiquemos los números nuevamente en una secuencia de texto con la instancia `Wav2Vec2tokenizer`, que definimos anteriormente.

In [ ]:
predictions = tf.argmax(outputs, axis=-1)
predictions = [tokenizer.decode(pred) for pred in predictions.numpy().tolist()]
predictions

Esta predicción es bastante aleatoria ya que el modelo nunca se entrenó con datos grandes en este cuaderno (pués este cuaderno no está diseñado para realizar un entrenamiento completo). Obtendrá buenas predicciones si entrena este modelo en el conjunto de datos completo de LibriSpeech.

Por fin llegamos al final de este cuaderno. Pero no es el final del aprendizaje de TensorFlow para tareas relacionadas con la voz; este [repositorio](https://github.com/tulasiram58827/TTS_TFLite) contiene más tutoriales increíbles. En caso de que encuentre algún error en este cuaderno, cree un problema [aquí](https://github.com/vasudevgupta7/gsoc-wav2vec2/issues).